# Building databases of published works  

> Pragmatic tools for constructing databases of scientific works based on queries defined with Boolean Logic.

In [ ]:
#| default_exp utils.db

In [1]:
#| hide
from nbdev import *

Tabulate queries in a spreadsheet and generate a database based on the data from those queries. 

**Example**:  Define a dataframe with an `id` column and a `query` column (expressing a search query in Boolean Logic):

| ID | DISEASE NAME | MONDO_ID | QUERY  | 
|----|--------------|----------|--------|
| 1 | Adult Polyglucosan Body Disease | MONDO:0009897 | adult polyglucosan body disease \| adult polyglucosan body neuropathy
| 2 | AGAT deficiency | MONDO:0012996 |  "GATM deficiency" \| "AGAT deficiency" \| "arginine:glycine amidinotransferase deficiency" \| "L-arginine:glycine amidinotransferase deficiency"
| 3 | Guanidinoacetate methyltransferase deficiency | MONDO:0012999 |  "guanidinoacetate methyltransferase deficiency" \| "GAMT deficiency"
| 4 | CLOVES Syndrome | MONDO:0013038 | "CLOVES syndrome \| (congenital lipomatous overgrowth) & (vascular malformation epidermal) & (nevi-spinal) & syndrome \| (congenital lipomatous overgrowth) & (vascular malformations) & (Epidermal nevi) & ((skeletal\|spinal) & abnormalities) \| CLOVE syndrome \| (congenital lipomatous overgrowth) & (vascular malformation) & (epidermal nevi)


In [1]:
#| export

from pathlib import Path
from alhazen.utils.airtableUtils import AirtableUtils
from alhazen.utils.searchEngineUtils import ESearchQuery, EuroPMCQuery
from alhazen.utils.queryTranslator import QueryTranslator, QueryType

import pandas as pd

from datetime import datetime
from time import time,sleep

import requests
import json
from tqdm import tqdm
import os
import re

In [5]:
#| export

class DashboardDb:

  """This class runs queries on external repositories to provide dataframes of linked corpora and papers that can be used to populate a local, medium scale database.

  Functionality includes:

    * Define a spreadsheet with a column of queries expressed in boolean logic
    * Optional: Define a secondary spreadsheet with a column of subqueries expressed in boolean logic
    * Iterate over different sources (Pubmed + European Pubmed) to execute all combinations of queries and subqueries
    
  """

  def __init__(self, name, description, loc):
    self.name = name
    self.description = description
    self.loc = loc

    if os.path.exists(loc) is False:
      os.mkdir(loc)

    log_path = '%s/db_log.txt' % (loc)
    if os.path.exists(log_path) is False:
      Path(log_path).touch()
      
  def execute_pubmed_queries(self, qt, qt2, api_key='', sections=['tiab']):
    corpus_paper_list = []
    errors = []
    (corpus_ids, pubmed_queries) = qt.generate_queries(QueryType.pubmed, sections=sections)
    if qt2:
      (subset_ids, pubmed_subset_queries) = qt2.generate_queries(QueryType.pubmed, sections=sections)
    else: 
      (subset_ids, pubmed_subset_queries) = ([0],[''])
    for (i, q) in zip(corpus_ids, pubmed_queries):
      for (j, sq) in zip(subset_ids, pubmed_subset_queries):
        query = q
        if query=='nan' or len(query)==0: 
          errors.append((i, j, query))
          continue
        if len(sq) > 0:
          query = '(%s) AND (%s)'%(q, sq) 
        pmq = ESearchQuery(api_key=api_key)
        num_found = pmq.execute_count_query(query)
        print(num_found)
        if num_found>0:
          pmids = pmq.execute_query(query)
          sleep(0.5) # Sleep for half a second
          for id in tqdm(pmids):
            corpus_paper_list.append((id, i, 'pubmed', j))
    return corpus_paper_list

  def execute_epmc_queries_on_sections(self, qt, qt2, sections=['paper_title', 'ABSTRACT'], 
                                       extra_columns=["id", "source", "doi", 'title', 'pubYear', 'abstractText', 'pubType']):
    corpus_paper_list = []
    epmc_errors = []
    (corpus_ids, epmc_queries) = qt.generate_queries(QueryType.epmc, sections=sections)
    if qt2:
      (subset_ids, epmc_subset_queries) = qt2.generate_queries(QueryType.epmc, sections=sections)
    else: 
      (subset_ids, epmc_subset_queries) = ([0],[''])
    for (i, q) in zip(corpus_ids, epmc_queries):
      for (j, sq) in zip(subset_ids, epmc_subset_queries):
        query = q
        if query is None or query=='nan' or len(query)==0: 
          continue
        if len(sq) > 0:
          query = '(%s) AND (%s)'%(q, sq) 
        epmcq = EuroPMCQuery()
        #try:
        numFound, epmc_pmids = epmcq.run_empc_query(query, extra_columns=extra_columns)
        for row in tqdm(epmc_pmids):
            tup = [row[0], i, 'epmc', j, row[1]]
            if len(row)>2:
                tup.extend(row[2:])
            corpus_paper_list.append(tup)
        #except Exception as e:
        #  epmc_errors.append((i, j, query, e))
    return corpus_paper_list, epmc_errors

  def check_query_terms(self, qt, qt2=None, pubmed_api_key=''):
    pmq = ESearchQuery(api_key=pubmed_api_key)
    terms = set()
    for t in qt.terms2id.keys():
        terms.add(t)
    if qt2 is not None:
        for t2 in qt2.terms2id.keys():
            terms.add(t2)
    check_table = {} 
    for t in tqdm(terms):
        (is_ok, t2, c) = pmq._check_query_phrase(t)
        check_table[t] = (is_ok, c)
    return check_table

  def airtable_to_corpus_dataframes(self, at_key, at_file, at_sheets, 
                                    extra_columns=["id", "source", "doi", 'title', 'pubYear', 'abstractText', 'pubType']):
    atu = AirtableUtils(at_key)
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    for sn, id_col, query_col, col_map, sections in at_sheets: 
        cdf = atu.read_airtable(at_file, sn)
        cdf = cdf.rename(columns={id_col:'ID', query_col:'QUERY'})
        cdf = cdf.rename(columns=col_map)
        cdf = cdf.fillna('').rename(
            columns={c:re.sub('[\s\(\)]','_', c.upper()) for c in cdf.columns}
            )
        cdf.QUERY = [re.sub('^http[s]*://', '', r.QUERY) if r.QUERY[:4]=='http' else r.QUERY 
                     for i,r in cdf.iterrows()] 
        cdf.QUERY = [re.sub('/$', '', r.QUERY.strip())  
                        for i,r in cdf.iterrows()] 
        df1 = pd.concat([df1, cdf])
        qt = QueryTranslator(cdf, 'ID', 'QUERY')
        paper_list, errors = self.execute_epmc_queries_on_sections(qt, None, sections=sections)
        l = [tup if re.match('\d',tup[0]) else (-1, tup[1], tup[2], tup[3], tup[4]) for tup in paper_list]
        cols = ['ID_PAPER', 'ID_CORPUS', 'SOURCE', 'SUBSET_CODE', 'DOI']
        cols.extend(extra_columns)
        temp = pd.DataFrame(paper_list, columns=cols)
        df2 = pd.concat([df2, temp.drop(columns=extra_columns).drop_duplicates()])
        if len(extra_columns)>0:
            df3 = pd.concat([df3, temp.drop(columns=['ID_CORPUS','SOURCE', 'SUBSET_CODE']).drop_duplicates()])
    df1.fillna('', inplace=True)
    df1 = df1.reset_index(drop=True)
    df2.fillna('', inplace=True)
    df2 = df2.reset_index(drop=True)
    df3.fillna('', inplace=True)
    df3 = df3.reset_index(drop=True).drop_duplicates()
    
    return df1, df2, df3
  
  def build_corpus_dataframes(self, cdf, id_col, query_col, col_map={}, sections=['TITLE','ABSTRACT'], 
                              extra_columns=["id", "source", "doi", 'title', 'pubYear', 'abstractText', 'pubType']):
    
    cdf = cdf.rename(columns={id_col:'ID', query_col:'QUERY'})
    cdf = cdf.rename(columns=col_map)
    cdf = cdf.fillna('').rename(
        columns={c:re.sub('[\s\(\)]','_', c.upper()) for c in cdf.columns}
        )
    cdf.QUERY = [re.sub('^http[s]*://', '', r.QUERY) if r.QUERY[:4]=='http' else r.QUERY 
                  for i,r in cdf.iterrows()] 
    cdf.QUERY = [re.sub('/$', '', r.QUERY.strip())  
                    for i,r in cdf.iterrows()] 
    qt = QueryTranslator(cdf, 'ID', 'QUERY')
    paper_list, errors = self.execute_epmc_queries_on_sections(qt, None, sections=sections)
    l = [tup if re.match('\d',tup[0]) else (-1, tup[1], tup[2], tup[3], tup[4]) for tup in paper_list]
    cols = ['ID_PAPER', 'ID_CORPUS', 'SOURCE', 'SUBSET_CODE', 'DOI']
    cols.extend(extra_columns)
    temp = pd.DataFrame(paper_list, columns=cols)
    df2 = temp.drop(columns=extra_columns).drop_duplicates()
    if len(extra_columns)>0:
        df3 = temp.drop(columns=['ID_CORPUS','SOURCE', 'SUBSET_CODE']).drop_duplicates()
    cdf.fillna('', inplace=True)
    df2.fillna('', inplace=True)
    df3.fillna('', inplace=True)
    return cdf, df2, df3
  

In [3]:
from io import StringIO

EM_QUERIES_TSV = '''
ID,NAME,QUERY
0,Hierarchical phase-contrast tomography,Hierarchical phase-contrast tomography | HIP-CT | Hierarchical phase contrast tomography
1,Cryo-Electron Tomography,Cryoelectron Tomography | Cryo Electron Tomography | Cryo-Electron Tomography | Cryo-ET | CryoET
2,Volume Electron Microscopy,Volume Electron Microscopy | Volume EM | (serial section & (electron microscopy | EM | transmission electron microscopy | TEM | scanning electron microscopy | SEM | electron tomography )) | (serial block-face & (SEM | scanning electron microscopy)) | (focused ion beam & (SEM | scanning electron microscopy)) | (automated serial & (TEM | transmission electron microscopy)) | ( massively parallel imaging & (SEM | scanning electron microscopy)) | multibeam SEM | FAST-SEM | cryo-TEM
'''
EM_QUERIES_TSV = '''
ID,NAME,QUERY
1,Cryo-Electron Tomography,Cryoelectron Tomography | Cryo Electron Tomography | Cryo-Electron Tomography | Cryo-ET | CryoET
'''

em_queries_df = pd.read_csv(StringIO(EM_QUERIES_TSV), sep=',')
em_queries_df

,ID,NAME,QUERY
0,1,Cryo-Electron Tomography,Cryoelectron Tomography | Cryo Electron Tomogr...


In [6]:
db = DashboardDb('em_literature', 'EuropePMC papers based on EM keywords', '/Users/gburns/Documents/2023H2/cryoet_portal_rocrates/EM_LITERATURE_DB')
df1, df2, df3 = db.build_corpus_dataframes(em_queries_df, 'ID', 'QUERY', sections=['TITLE','ABSTRACT', 'METHODS'])

100%|██████████| 1/1 [00:00<00:00, 1101.16it/s]


https://www.ebi.ac.uk/europepmc/webservices/rest/search?format=JSON&pageSize=1000&synonym=TRUE&resultType=core&query=((TITLE:"Cryoelectron Tomography" OR ABSTRACT:"Cryoelectron Tomography" OR METHODS:"Cryoelectron Tomography") OR (TITLE:"Cryo Electron Tomography" OR ABSTRACT:"Cryo Electron Tomography" OR METHODS:"Cryo Electron Tomography") OR (TITLE:"Cryo-Electron Tomography" OR ABSTRACT:"Cryo-Electron Tomography" OR METHODS:"Cryo-Electron Tomography") OR (TITLE:"Cryo-ET" OR ABSTRACT:"Cryo-ET" OR METHODS:"Cryo-ET") OR (TITLE:"CryoET" OR ABSTRACT:"CryoET" OR METHODS:"CryoET")), 2431 European PMC PAPERS FOUND


100%|██████████| 3/3 [00:35<00:00, 11.73s/it]


 Returning 2431


100%|██████████| 2431/2431 [00:00<00:00, 287723.72it/s]


In [9]:
df3

,ID_PAPER,DOI,id,source,doi,title,pubYear,abstractText,pubType
0,10620151,10.1046/j.1365-2818.2000.00629.x,10620151,MED,10.1046/j.1365-2818.2000.00629.x,Soft X-ray microscopy with a cryo scanning tra...,2000,Using a cryo scanning transmission X-ray micro...,
1,10675296,10.1006/jsbi.1999.4204,10675296,MED,10.1006/jsbi.1999.4204,Cryo-electron tomography of neurospora mitocho...,2000,Cryo-electron tomography was used to study the...,
2,10806087,10.1006/jsbi.2000.4215,10806087,MED,10.1006/jsbi.2000.4215,The cell surface glycoprotein layer of the ext...,2000,We have studied the surface layer (S-layer) of...,
3,11516947,10.1016/s0960-9822(01)00349-9,11516947,MED,10.1016/s0960-9822(01)00349-9,FhuA-mediated phage genome transfer into lipos...,2001,<h4>Background</h4>The transfer of phage genom...,
4,12160700,10.1016/s1047-8477(02)00020-5,12160700,MED,10.1016/s1047-8477(02)00020-5,Use of frozen-hydrated axonemes to assess imag...,2002,"Using a 400-kV cryoelectron microscope, we hav...",
...,...,...,...,...,...,...,...,...,...
2426,PPR93207,10.1101/777854,PPR93207,PPR,10.1101/777854,Boiling Acid Mimics Intracellular Giant Virus ...,2019,"<h4>Summary</h4> Since their discovery, giant ...",
2427,PPR94004,10.1101/783373,PPR94004,PPR,10.1101/783373,Crystal structure of human PACRG in complex wi...,2019,"In human, the Parkin Co-Regulated Gene (PACRG)...",
2428,PPR94275,10.1101/786715,PPR94275,PPR,10.1101/786715,Bacterial flagellar motor PL-ring disassembly ...,2019,The bacterial flagellar motor is an amazing na...,
2429,PPR95335,10.1101/797514,PPR95335,PPR,10.1101/797514,"Fully automated, sequential focused ion beam m...",2019,Cryo-electron tomography (cryoET) has become a...,


In [17]:
!test -f /tmp/tmp.db && rm /tmp/tmp.db

In [21]:
!cat ../sql/schema.sql | sqlite3 /tmp/alhazen/tmp.db 

In [33]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt

def mm(graph):
  graphbytes = graph.encode("ascii")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  display(
    Image(
      url="https://mermaid.ink/img/"
      + base64_string
    )
  )



In [8]:
from alhazen.schema_sqla import Work
help(Work)

Work(id=100, title='Test', abstract='Test abstract', iri='10.1234/1234')

Help on class Work in module alhazen.schema_sqla:

class Work(InformationContentEntity)
 |  Work(**kwargs)
 |  
 |  A published work
 |  
 |  Method resolution order:
 |      Work
 |      InformationContentEntity
 |      NamedThing
 |      Entity
 |      sqlalchemy.orm.decl_api.Base
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **kwargs)
 |      A simple constructor that allows initialization from kwargs.
 |      
 |      Sets attributes on the constructed instance using the names and
 |      values in ``kwargs``.
 |      
 |      Only keys that are present as
 |      attributes of the instance's class are allowed. These could be,
 |      for example, any mapped columns or relationships.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  abstract
 |  
 |  authors
 |  
 |  creation_date
 |  
 |  format
 |  
 |  full_text
 |  
 |  ha

Work(id=100,title=Test,abstract=Test abstract,full_text=None,license=None,rights=None,format=None,creation_date=None,name=None,iri=10.1234/1234,)

In [10]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [28]:
engine = create_engine("sqlite:////tmp/alhazen/tmp.db")
session_class = sessionmaker(bind=engine)
session = session_class()
p = Work(id=100, title='Test', abstract='Test abstract', iri='10.1234/1234')
session.add(p)
session.commit()

In [34]:
!rm /tmp/alhazen/tmp.db

In [29]:
!sqlite3 /tmp/alhazen/tmp.db "SELECT * FROM Work;" ".exit"

100|Test|Test abstract|||||||10.1234/1234


In [32]:
for w in session.query(Work).where(Work.id!=100):
    print(p.title)
